<a href="https://colab.research.google.com/github/YashwardhanSingh2005/PytorchLearning/blob/main/Pytorch_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inserting Packages


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

# Setting up Dataloaders and running script


In [2]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [9]:
def check_image(path):
    try:
        im = Image.open(path)
        return im.format in ["JPEG", "PNG", "JPG"]
    except:
        return False

In [8]:
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [10]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [11]:
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

In [12]:
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# My first Pytorch NN model

In [14]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
simplenn = SimpleNet()

In [16]:
optimizer = optim.Adam(simplenn.parameters(), lr=0.001)

In [15]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [17]:
train(simplenn, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10)

<ipython-input-15-f38d79bc5988>:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)


Epoch: 0, Training Loss: 3.23, Validation Loss: 3.14, accuracy = 0.32
Epoch: 1, Training Loss: 2.04, Validation Loss: 2.07, accuracy = 0.64
Epoch: 2, Training Loss: 1.38, Validation Loss: 0.65, accuracy = 0.71
Epoch: 3, Training Loss: 0.52, Validation Loss: 1.06, accuracy = 0.64
Epoch: 4, Training Loss: 0.57, Validation Loss: 0.64, accuracy = 0.72
Epoch: 5, Training Loss: 0.38, Validation Loss: 0.69, accuracy = 0.72
Epoch: 6, Training Loss: 0.32, Validation Loss: 0.77, accuracy = 0.65
Epoch: 7, Training Loss: 0.31, Validation Loss: 0.68, accuracy = 0.70
Epoch: 8, Training Loss: 0.25, Validation Loss: 0.74, accuracy = 0.68
Epoch: 9, Training Loss: 0.23, Validation Loss: 0.74, accuracy = 0.68


In [21]:
labels = ['cat','fish']

img = Image.open("./val/fish/100_1422.JPG")
img = img_transforms(img)
prediction = F.softmax(simplenn(img))
print(prediction)
prediction = prediction.argmax()
print(labels[prediction])

tensor([[0.0016, 0.9984]], grad_fn=<SoftmaxBackward0>)
fish


<ipython-input-21-1c2c39363c60>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prediction = F.softmax(simplenn(img))
